In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql

In [ ]:
# connect to SQLite database
db_path = "../DataTables/basketball.sqlite"
connection = sql.connect(db_path) # create connection object to database
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

In [ ]:
biosQ = """SELECT * FROM 'Player_Attributes'"""

bios_df = pd.read_sql(biosQ, connection)
bios_df.tail()

In [ ]:
injuries_df = pd.read_csv('../DataTables/Injuries - KAggle.csv')
injuries_df.head()

In [ ]:
injuries_df["Key"] = injuries_df["Key"].str.replace(".","")
injuries_df["Relinquished"] = injuries_df["Relinquished"].str.replace(".","")
injuries_df.head()

In [ ]:
injuries_pivot = pd.pivot_table(injuries_df, values='Notes', index=['Key', 'Team','Relinquished'],
                    aggfunc='count')
injuries_pivot.head()

In [ ]:
injuries_pivot.reset_index(inplace=True)

In [ ]:
injuries_pivot.head()

In [ ]:
team_Q = """SELECT * FROM 'Team'"""

teams_df = pd.read_sql(team_Q, connection)
teams_df.head()

In [ ]:
teams_df.columns

In [ ]:
injuries_team=pd.merge(injuries_pivot,teams_df[['id','full_name','abbreviation','nickname']],how="left",left_on='Team',right_on='nickname').fillna(np.nan)
injuries_team.head()

In [ ]:
injuries_pivot.shape

In [ ]:
injuries_team.shape

In [ ]:
injuries_team=injuries_team.drop(['Team','nickname'],axis=1)

In [ ]:
injuries_team.head()

In [ ]:
injuries_team["NewKey"]=injuries_team["Key"]+injuries_team["abbreviation"]
injuries_team.head()

In [ ]:
injuries_team=injuries_team.drop(['Key'],axis=1)
injuries_team.head()

In [ ]:
injuries_team=injuries_team.rename(columns={"Relinquished": "Player", "Notes": "Injuries","full_name":"TeamName","abbreviation":"TeamCode",
                             "NewKey":"Key","id": "TeamId"})
injuries_team.head()

In [ ]:
injuries_team['DraftYear']=np.nan
injuries_team.head()

In [ ]:
# capturing Injury year from key
injuries_team['DraftYear'] = injuries_team['Key'].str[:4]
injuries_team.head()    

In [ ]:
playersQ = """SELECT * FROM 'Player'"""

players_df = pd.read_sql(playersQ, connection)
players_df.head()

In [ ]:
injuries_team_player = injuries_team.merge(players_df[['id','full_name']],how='left',left_on='Player',right_on='full_name').fillna(np.nan)
injuries_team_player.head()

In [ ]:
injuries_team_player.shape

In [ ]:
injuries_team_player=injuries_team_player.drop(['full_name'],axis=1)
injuries_team_player=injuries_team_player.rename(columns={"id": "PlayerId"})

In [ ]:
injuries_team_player=injuries_team_player.drop(['Key'],axis=1)

In [ ]:
injuries_team_player.head()

## Bring in All Seasons data to get points

In [ ]:
seasons_df = pd.read_csv('../DataTables/AllSeasons.csv')
seasons_df.head()

In [ ]:
seasons_df["player_name"] = seasons_df["player_name"].str.replace(".","")

In [ ]:
seasons_df_clean = seasons_df[seasons_df['draft_year'] != 'Undrafted']
seasons_df_clean.head()

In [ ]:
seasons_df_clean = seasons_df_clean[seasons_df_clean['draft_year'] >= 2010]
seasons_df_clean.head()

In [ ]:
seasons_df_clean['draft_year'] = seasons_df_clean['draft_year'].astype(str)

In [ ]:
max(seasons_df_clean['draft_year'])

In [ ]:
seasons_df_clean['Key']= seasons_df_clean['player_name'] + seasons_df_clean['team_abbreviation']+seasons_df_clean['draft_year']
seasons_df_clean.head()

In [ ]:
injuries_team_player['Key'] = injuries_team_player['Player']+injuries_team_player['TeamCode']+injuries_team_player['DraftYear']
injuries_team_player.head()

In [ ]:
seasons_df_clean.columns

In [ ]:
injuries_team_player.columns

In [ ]:
injuries_points = seasons_df_clean.merge(injuries_team_player[['Key','Injuries','PlayerId','TeamId']],how='left',on='Key').fillna(0)

In [ ]:
injuries_points.shape

In [ ]:
injuries_points.head()

In [ ]:
injuries_points.columns

In [ ]:
injuries_points=injuries_points.drop(['age', 'player_height', 'player_weight','college', 'country',
                       'draft_round','draft_number', 'net_rating', 'oreb_pct',
                       'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct', 'season'],axis=1)

In [ ]:
injuries_points.head()

In [ ]:
injuries_points.shape

In [ ]:
seasons_df_clean[seasons_df_clean['Key']=='Maurice EvansWAS2010']

## Update playerID and Team ID for missing ones

In [ ]:
players_Q = """SELECT * FROM 'Player'"""

players_df = pd.read_sql(players_Q, connection)
players_df.head()

In [ ]:
players_df["full_name"] = players_df["full_name"].str.replace(".","")

In [ ]:
players_df = players_df[players_df['id'].map(len)>=4]
players_df.head()

In [ ]:
players_df2=players_df[(players_df.duplicated(subset = ['full_name'])==True)]
players_df2.head()

In [ ]:
#players_df.to_csv('../DataTables/tokeep.csv',index=False)
players_df2.to_csv('../DataTables/todelete.csv',index=False)

In [ ]:
#removed the duplicate players from the 2st file based on the 2nd file
players_df=pd.read_csv('../DataTables/tokeep.csv')
players_df.shape

In [ ]:
player_injuries=injuries_points.merge(players_df[['full_name','id']],how='left',left_on='player_name',right_on='full_name').fillna(np.nan)
player_injuries.head()

In [ ]:
player_injuries.shape

In [ ]:
injuries_points.shape

In [ ]:
player_injuries.head()

In [ ]:
#drop full name and delete playerID with blank id, and then rename id to player id
clean_injuries1=player_injuries.drop(['full_name','PlayerId'],axis=1)
clean_injuries2=clean_injuries1.rename(columns = {'id':'PlayerId'})
clean_injuries2['PlayerId']=clean_injuries2['PlayerId'].astype(str)
clean_injuries2['PlayerId']=clean_injuries2['PlayerId'].str.replace("\.0","")
clean_injuries2.head()

In [ ]:
clean_injuries2.shape

In [ ]:
teams_df.head()

In [ ]:
clean_injuries=clean_injuries2.merge(teams_df[['id','abbreviation']],how="left",left_on='team_abbreviation',right_on='abbreviation')
clean_injuries.shape

In [ ]:
clean_injuries.head()

In [ ]:
clean_injuries=clean_injuries.drop(['abbreviation','TeamId'],axis=1)
clean_injuries=clean_injuries.rename(columns = {'id':'TeamId'})
clean_injuries.head()

In [ ]:
clean_injuries.shape

In [ ]:
min(clean_injuries['draft_year'])

In [ ]:
max(clean_injuries['draft_year'])

## Get birthdate from player attributes table

In [ ]:
playersAttr_Q = """SELECT * FROM 'Player_Attributes'"""

playersAttr_df = pd.read_sql(playersAttr_Q, connection)
playersAttr_df.head()

In [ ]:
playersAttr_df.dtypes

In [ ]:
injuries_all_up=clean_injuries.merge(playersAttr_df[['ID','BIRTHDATE']],how="left",left_on='PlayerId',right_on='ID')
injuries_all_up.shape

In [ ]:
injuries_all_up.head()

In [ ]:
injuries_all_up['BIRTHDATE']=pd.to_datetime(injuries_all_up['BIRTHDATE'])
injuries_all_up.dtypes

In [ ]:
from datetime import date
def calculateAge(birthDate):
    today = date.today()
    age = today.year - birthDate.year -((today.month, today.day) < (birthDate.month, birthDate.day))
 
    return age

In [ ]:
for i,row in injuries_all_up.iterrows():
    age=calculateAge(row['BIRTHDATE'])
    injuries_all_up.loc[i,'Age']=age

In [ ]:
injuries_all_up.head()

In [ ]:
injuries_all_up.shape

In [ ]:
injuries_all_up.drop(['ID'],axis=1)

In [ ]:
injuries_all_up.to_csv("../DataTables/Injuries2010-2020.csv")

## code to update the missing birthdays

In [ ]:
injuries_all_up.head()

In [ ]:
bday_miss=injuries_all_up[injuries_all_up['BIRTHDATE'].isnull()]
bday_miss.head()

In [ ]:
playersAttr_df.columns

In [ ]:
bday_update = bday_miss.merge(playersAttr_df[['ID','DISPLAY_FIRST_LAST','BIRTHDATE']],how="left",left_on='player_name',right_on='DISPLAY_FIRST_LAST')
bday_update

In [ ]:
bday_update['ID_x'].isna().sum()

In [ ]:
bday_update.count()

In [ ]:
bday_update=bday_update.drop(['PlayerId','DISPLAY_FIRST_LAST','ID_x','BIRTHDATE_x'],axis=1)

In [ ]:
bday_update.head()

In [ ]:
bday_update=bday_update.rename(columns = {'ID_y':'PlayerId','BIRTHDATE_y':'BIRTHDATE'})

In [ ]:
bday_update[bday_update['PlayerId'].isnull()]

In [ ]:
playersAttr_df[playersAttr_df['DISPLAY_FIRST_LAST']=='Charlie Brown Jr']

In [ ]:
bday_update.loc[bday_update['player_name']=='Danuel House','PlayerId']='1627863'

In [ ]:
bday_update[bday_update['player_name']=='Marcus Morris']

In [ ]:
bday_update.loc[bday_update['PlayerId']=='1627863','BIRTHDATE_y']='1993-06-07T00:00:00'

In [ ]:
null_players = pd.read_csv('../DataTables/NullPlayers.csv')
null_players

In [ ]:
bday_update=bday_update[bday_update['BIRTHDATE'].isnull()]
bday_update.head()

In [ ]:
bday_update = bday_update.merge(null_players[['player_name','BIRTHDATE']],how="left",left_on='player_name',right_on='player_name')
bday_update.tail()

In [ ]:
bday_update=bday_update.drop(['BIRTHDATE_x'],axis=1)
bday_update.head()

In [ ]:
bday_update.shape

In [ ]:
bday_update=bday_update.drop(['team_abbreviation_x', 'draft_year_x', 'gp_x', 'pts_x',
       'reb_x', 'ast_x', 'Key_x', 'Injuries_x', 'TeamId_x', 'PlayerId_x',
       'BIRTHDATE_x'],axis=1)
bday_update

In [ ]:
bday_update.head()

In [ ]:
bday_update=bday_update.rename(columns = {'BIRTHDATE_y':'BIRTHDATE'})

In [ ]:
injuries_all_up.columns

In [ ]:
injuries_all_up2.shape

In [ ]:
injuries_all_up2 = injuries_all_up.merge(bday_update[['Key','PlayerId','BIRTHDATE']],how='left',on='Key')
injuries_all_up2.head()

In [ ]:
for i,row in injuries_all_up2.iterrows():
    if row['PlayerId_x']=='nan':
        pid = row['PlayerId_y']
        injuries_all_up2.loc[i,'PlayerId_x']=pid
        
   # if pd.isnull(row['BIRTHDATE_x']):
   #     bday = row['BIRTHDATE_y']
   #     injuries_all_up2.loc[i,'BIRTHDATE_x']=bday

injuries_all_up2.head()

In [ ]:
null_b=injuries_all_up2[injuries_all_up2['BIRTHDATE_x'].isnull()]

In [ ]:
null_b.to_csv('../DataTables/nullbirthdays.csv',index=False)

In [ ]:
playersAttr_df[playersAttr_df['DISPLAY_FIRST_LAST']=='Michael Frazier II']

In [ ]:
null_b=null_b.merge(playersAttr_df[['ID','DISPLAY_FIRST_LAST','BIRTHDATE']],how="left",left_on='player_name',right_on='DISPLAY_FIRST_LAST')
null_b.head()

In [ ]:
null_b.columns

In [ ]:
null_b=null_b.drop(['BIRTHDATE_x','DISPLAY_FIRST_LAST'],axis=1)

In [ ]:
null_b.head()

In [ ]:
null_b[null_b['ID_x'].notnull()]

In [ ]:
null_b.isna().sum()

In [ ]:
null_b=null_b.drop(['PlayerId_x','ID_x','PlayerId_y','BIRTHDATE_y','Age'],axis=1)

In [ ]:
injuries_all_up3=injuries_all_up2.merge(null_b[['Key','BIRTHDATE','ID_y']],how='left',on='Key')
injuries_all_up3.head()

In [ ]:
injuries_all_up3.isna().sum()

In [ ]:
injuries_all_up3.shape

In [ ]:
injuries_all_up3.to_csv('../DataTables/Injuries_all_up.csv',index=True)

## Did some final clean-up in excel and renamed the file to 'Games and injuries stats-2010-2020.csv'